In [1]:
# In the beginning let's install transformers library
!pip install transformers

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import torch
from tqdm import tqdm, trange
import numpy as np


from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import CamembertTokenizer, CamembertForSequenceClassification
from transformers import AdamW
# import torch.optim as optim
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# import seaborn as sns

In [3]:
# Defining constants
epochs = 3#0
MAX_LEN = 512#256#128
batch_size = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
df = pd.read_csv("../input/sport-fr/eurosport_fr.csv")

df['category'].replace({'formule-1':"course", 'auto-moto':"course", 'motocyclisme':"course"}, inplace=True)
df.category.value_counts()

df['category_id'] = df['category'].factorize()[0]
df = df.loc[df.text.notnull()].reset_index(drop=True)
print(len(df.category.value_counts()))
df.head()

18


,url,title,sub_title,img_src,text,category,category_id
0,https://www.eurosport.fr//formule-1/michael-n-...,"""Michael n'était pas très fort pour livrer une...",FORMULE 1 - Mattia Binotto fut l'une des rares...,https://imgresizer.eurosport.com/unsafe/1200x0...,Jean Todt avait jeté son dévolu sur Michael Sc...,course,0
1,https://www.eurosport.fr//formule-1/saison-202...,Saison 2021 de Formule 1 : Le calendrier confi...,SAISON 2021 - A l'occasion du Conseil Mondial ...,https://imgresizer.eurosport.com/unsafe/1200x0...,"Si les conditions sanitaires le permettent, vi...",course,0
2,https://www.eurosport.fr//formule-1/saison-202...,Transferts - AlphaTauri met fin au suspens : Y...,SAISON 2021 - Ce n'était un secret pour person...,https://imgresizer.eurosport.com/unsafe/1200x0...,"C’était un secret de Polichinelle, Red Bull l’...",course,0
3,https://www.eurosport.fr//formule-1/formule-1-...,Formule 1 - Rebondissement de dernière minute ...,FORMULE 1 - Selon les informations du quotidie...,https://imgresizer.eurosport.com/unsafe/1200x0...,Soirée mouvementée du côté du paddock. Alors q...,course,0
4,https://www.eurosport.fr//formule-1/frank-will...,Frank Williams hospitalisé,L'écurie Williams a fait savoir que Sir Frank ...,https://imgresizer.eurosport.com/unsafe/1200x0...,L'année 2020 n'est pas de tout repos pour la f...,course,0


# Spliting Training and validation data

In [5]:
# Initialize CamemBERT tokenizer
tokenizer = CamembertTokenizer.from_pretrained('camembert-base',do_lower_case=True)

In [6]:
# Creates list of texts and labels
#text = df['text'].to_list()
#labels = df['label'].to_list()

# Creates list of texts and labels
text = df['text'].to_list()
labels = df['category_id'].to_list()

#user tokenizer to convert sentences into tokenizer
input_ids  = [tokenizer.encode(sent,add_special_tokens=True,max_length=MAX_LEN) for sent in text]

# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]  
    attention_masks.append(seq_mask)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [7]:
train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks = train_test_split(input_ids, labels, attention_masks,
                                                            random_state=42, test_size=0.1)

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)



train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

# CamemBERT model

In [8]:
model = CamembertForSequenceClassification.from_pretrained("camembert-base", num_labels=18)#20
model.to(device)

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.weight'

CamembertForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Laye

# Defining the parameters and metrics to optimize

In [9]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, eps=10e-8)#2e-5

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# Training and evaluating our model

In [ ]:
# Store our loss and accuracy for plotting if we want to visualize training evolution per epochs after the training process
train_loss_set = []

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):  
    # Tracking variables for training
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
  
    # Train the model
    model.train()
    for step, batch in enumerate(train_dataloader):
        # Add batch to device CPU or GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()
        # Forward pass
        outputs = model(b_input_ids,token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        # Get loss value
        loss = outputs[0]
        # Add it to train loss list
        train_loss_set.append(loss.item())    
        # Backward pass
        loss.backward()
        # Update parameters and take a step using the computed gradient
        optimizer.step()
    
        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    


    # Tracking variables for validation
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    # Validation of the model
    model.eval()
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Add batch to device CPU or GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions
            outputs =  model(b_input_ids,token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            loss, logits = outputs[:2]
    
        # Move logits and labels to CPU if GPU is used
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
torch.save(model.state_dict(), f"model_fr.bin")

In [ ]:

news = ["La Sud-Coréenne Kim A-lim a remporté sa première grande victoire internationale en réalisant un impressionnant retour dans le dernier tour de l\'US Open féminin, lundi à Houston, au Texas.Kim A-lim, âgée de 25 ans, n'avait jamais gagné de tournoi en dehors du circuit sud-coréen. A Houston, elle a fini magnifiquement avec trois birdies pour s\'offrir son premier majeur.Lors du dernier tour, elle a rendu une carte de 67, soit quatre sous le par, pour finir à -3 au score global du tournoi. Elle dépasse sa compatriote et numéro 1 mondiale Ko Jin-young et l'Américaine Amy Olson, qui complète le podium.Le 75e US Open féminin avait été reporté de juin à décembre en raison de la pandémie de coronavirus, et se déroulait à huis clos depuis jeudi, pour les mêmes raisons. Le 4e et dernier tour, prévu dimanche, a été repoussé à lundi en raison du risque d'orage sur le parcours du Champions Golf Club à Houston."]

In [ ]:
# Encode the news
tokenized_comments_ids = [tokenizer.encode(comment,add_special_tokens=True,max_length=MAX_LEN) for comment in comments]
# Pad the resulted encoded news
tokenized_comments_ids = pad_sequences(tokenized_comments_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks 
attention_masks = []
for seq in tokenized_comments_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

prediction_inputs = torch.tensor(tokenized_comments_ids)
prediction_masks = torch.tensor(attention_masks)

In [ ]:
# Apply the finetuned model (Camembert)
flat_pred = []
with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs =  model(prediction_inputs.to(device),token_type_ids=None, attention_mask=prediction_masks.to(device))
    logits = outputs[0]
    logits = logits.detach().cpu().numpy() 
    flat_pred.extend(np.argmax(logits, axis=1).flatten())

In [ ]:
for i in range(len(flat_pred)):
    print('Comment: ', news[i])
    print('Label', flat_pred[i])

In [ ]:
df.loc[df['category_id'] == 8]